In [1]:
# add for tensorboard code
from __future__ import absolute_import, division, print_function, unicode_literals
import argparse
import json
from tensorflow.python.lib.io import file_io

# origin fairing code
import tensorflow as tf
import os
from tensorflow.python.keras.callbacks import Callback

class MyFashionMnist(object):
  def train(self):
    
    # argument
    parser = argparse.ArgumentParser()
    parser.add_argument('--tb_log_dir', default='./data/logs', type=str)
    args = parser.parse_args()
    tb_log_dir = args.tb_log_dir

    print("TensorFlow version: ", tf.__version__)

    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()

    callbacks = [tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir)]
    print("Training...")
    
    model.fit(x_train, y_train, epochs=3, validation_split=0.2, callbacks=callbacks) # here epochs
    
    score = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
    print('Test accuracy: ', score[1])

    # tensorboard
    metadata = {
        'outputs': [{
            'type': 'tensorboard',
            'source': tb_log_dir,
        }]
    }
    
    with file_io.FileIO('/mlpipeline-ui-metadata.json', 'w') as f:
        json.dump(metadata, f)

if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils
        
        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'   # 프라이빗 레지스트리

        fairing.config.set_builder(
            'append',
            image_name='tensorboard-job', # here not fairing job but katib job
            base_image='brightfly/kubeflow-jupyter-lab:tf2.0-cpu',
            registry=DOCKER_REGISTRY, 
            push=True
        )
        # cpu 1, memory 5GiB
        fairing.config.set_deployer('job',
                                    namespace='admin', # here
                                    pod_spec_mutators=[
                                        k8s_utils.get_resource_mutator(cpu=1,  # here
                                                                       memory=5)]
         
                                   )
        fairing.config.run()
    else:
        remote_train = MyFashionMnist()
        remote_train.train()

[I 201016 02:58:38 config:123] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7f4384376240>
[I 201016 02:58:38 config:125] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f433e27b780>
[I 201016 02:58:38 config:127] Using deployer: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f433e27b780>
[W 201016 02:58:38 append:50] Building image using Append builder...
[I 201016 02:58:38 base:105] Creating docker context: /tmp/fairing_context_ppx66kw1
[I 201016 02:58:38 converted_notebook:127] Converting 14-tensorboard-fairing.ipynb to 14-tensorboard-fairing.py
[I 201016 02:58:38 docker_creds_:234] Loading Docker credentials for repository 'brightfly/kubeflow-jupyter-lab:tf2.0-cpu'
[W 201016 02:58:40 append:54] Image successfully built in 2.415544261000832s.
[W 201016 02:58:40 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/tensorboard-job:6231543E...
[

TensorFlow version:  2.0.0
11493376/11490434 [==============================] - 0s 0us/step
2020-10-16 02:58:45.631482: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-10-16 02:58:45.638864: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000125000 Hz
2020-10-16 02:58:45.639436: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x47c2c40 executing computations on platform Host. Devices:
2020-10-16 02:58:45.639476: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
2020-10-16 02:58:46.672237: I tensorflow/core/profiler/lib/profiler_session.cc:184] Profiler session started.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
flatten (Flatten)            (None, 784)               0
________________

[W 201016 02:59:24 job:162] Cleaning up job fairing-job-dqcb4...
